In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Assessing "All Spas" Tab

This tab includes the serial#, model, and completion datetime for all spas manufactured from 2020 - Nov 22 2022

## Issues Identified:
- Time Description has ~ 12k missing values, and some are labeled "No Data"
- Serial# is of the object datatype. It might need to be converted to integer.
- Year and Month columns came from previous assessment. Those will be dropped.
- There are many duplicate serial#'s as expected. The most recent serial# should be kept. 

In [2]:
df = pd.read_excel('Blister Returns.xlsx', sheet_name = 'All Spas')

In [3]:
df.shape

(82165, 6)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82165 entries, 0 to 82164
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   serial#           82165 non-null  object        
 1   model             82165 non-null  object        
 2   CompleteDate      82165 non-null  datetime64[ns]
 3   Year              82165 non-null  int64         
 4   Month             82165 non-null  int64         
 5   Time Description  69915 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 3.8+ MB


In [5]:
print('Minimum Date', df['CompleteDate'].min())
print('Max Date', df['CompleteDate'].max())

Minimum Date 2020-03-02 06:19:00
Max Date 2022-11-22 13:12:33


In [6]:
duplicate_serial = df[df.duplicated('serial#')]

In [7]:
df['model'].value_counts()

X8        6733
A8        6221
R7        6026
A7L       5664
R6L       5251
R7L       5217
X7        5187
A7        5163
X6L       3820
R6        2879
R5L       2784
A6L       2754
X7L       2715
A9L       2330
A8L       2284
A6        1742
A5L       1686
M9        1608
STIL7     1489
R8        1415
M8        1402
A8D       1372
X5L       1363
M7        1304
X8L       1240
STIL5      687
X6R        674
R8L        569
S150       303
M6         278
S200         4
TRITON       1
Name: model, dtype: int64

In [8]:
df['Time Description'].value_counts()

6AM-Noon        18544
Noon-6PM        17448
6PM-Midnight    17377
Midnight-6AM    16411
No Data           135
Name: Time Description, dtype: int64

# Cleaning All Spas

In [9]:
df_clean = df.copy()

## Issue: There are many duplicate serial#'s as expected. The most recent serial# should be kept. 

## Resolution: Remove duplicates but keep the most recent version

In [10]:
df_clean.head(1)

,serial#,model,CompleteDate,Year,Month,Time Description
0,235854,A5L,2020-03-02 06:19:00,2020,3,NaN


In [11]:
df_clean = df_clean.sort_values('CompleteDate').drop_duplicates('serial#', keep='last')

### Test: Make sure duplicates were removed

In [12]:
duplicated_serial = df_clean[df_clean.duplicated('serial#')]
duplicated_serial

,serial#,model,CompleteDate,Year,Month,Time Description


## Issue: Year and Month columns came from previous assessment. Those will be dropped.

## Resolution: Drop these columns from dataframe

In [13]:
df_clean.drop(['Year', 'Month'], axis=1, inplace=True)

### Test: Make sure the columns were dropped successfully

In [14]:
df_clean.head()

,serial#,model,CompleteDate,Time Description
0,235854,A5L,2020-03-02 06:19:00,NaN
1,235905,A9L,2020-03-02 06:33:00,NaN
2,235881,A5L,2020-03-02 06:47:00,NaN
3,235873,A5L,2020-03-02 06:56:00,NaN
4,235891,A7,2020-03-02 07:15:00,NaN


## Issue: Time Description has ~ 12k missing values, and some are labeled "No Data"

## Resolution: Rows w/ missing values and/or valued at "No Data" will be removed from the dataset

In [15]:
df_clean.dropna(inplace=True)

In [16]:
df_clean = df_clean[df_clean['Time Description'] != 'No Data']

### Test: Making sure all null values and "No Data" values are removed from the dataframe

In [17]:
df_clean['Time Description'].value_counts()

6AM-Noon        16199
6PM-Midnight    15107
Noon-6PM        15094
Midnight-6AM    13594
Name: Time Description, dtype: int64

In [18]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59994 entries, 71 to 80242
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   serial#           59994 non-null  object        
 1   model             59994 non-null  object        
 2   CompleteDate      59994 non-null  datetime64[ns]
 3   Time Description  59994 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 2.3+ MB


### Issue: Serial# is of the object datatype. It might need to be converted to integer.
### Resolution: Converting serial# to integer datatype

In [19]:
df_clean['serial#'] = pd.to_numeric(df_clean['serial#'])

### Test: Make sure the datatype is now of the integer type

In [20]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59994 entries, 71 to 80242
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   serial#           59994 non-null  int64         
 1   model             59994 non-null  object        
 2   CompleteDate      59994 non-null  datetime64[ns]
 3   Time Description  59994 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 2.3+ MB


# Assessing Spray Info Data
This dataframe will include data from the spray automation machine for each spa 

## Issues Identified:
- Mix Module column should be removed. It has many null values
- columns should be renamed to avoid ambiguity
- there are many duplicate serial#

In [21]:
df1 = pd.read_excel('Blister Returns.xlsx', sheet_name='Spray Info')
df1.head()

,serial#,start_time,end_time,mix_module,operator,iso_press_avg,poly_press_avg,t_stamp,booth,iso_lbs_run_usage,poly_lbs_run_usage,Time
0,13456,2020-03-20 15:22:31.517,2020-03-20 15:37:35.463,88,admin,0.0,NaN,2020-03-20 15:37:36.070,3.0,1251.0,1500.0,0.651105
1,235970,2020-03-30 12:23:23.237,2020-03-30 12:59:49.633,88,admin,0.0,NaN,2020-03-30 12:59:50.300,3.0,1251.0,1500.0,0.541547
2,236746,2020-03-30 15:22:19.347,2020-03-30 16:36:38.323,88,admin,0.0,NaN,2020-03-30 16:36:39.033,3.0,1251.0,1500.0,0.692110
3,236730,2020-03-31 06:08:12.733,2020-03-30 19:14:41.770,88,admin,0.0,NaN,2020-03-31 06:37:54.207,3.0,1251.0,1500.0,0.801872
4,236771,2020-03-31 09:53:41.243,2020-03-31 06:52:04.443,88,admin,0.0,NaN,2020-03-31 09:56:50.683,3.0,1251.0,1500.0,0.286163


In [22]:
df1.shape

(63843, 12)

In [23]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63843 entries, 0 to 63842
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   serial#             63808 non-null  object        
 1   start_time          63752 non-null  datetime64[ns]
 2   end_time            63759 non-null  datetime64[ns]
 3   mix_module          24975 non-null  object        
 4   operator            63752 non-null  object        
 5   iso_press_avg       63648 non-null  float64       
 6   poly_press_avg      63478 non-null  float64       
 7   t_stamp             63757 non-null  datetime64[ns]
 8   booth               63757 non-null  float64       
 9   iso_lbs_run_usage   63051 non-null  float64       
 10  poly_lbs_run_usage  63051 non-null  float64       
 11  Time                63843 non-null  float64       
dtypes: datetime64[ns](3), float64(6), object(3)
memory usage: 5.8+ MB


In [24]:
duplicate_serial = df1[df1.duplicated('serial#')]
duplicate_serial

,serial#,start_time,end_time,mix_module,operator,iso_press_avg,poly_press_avg,t_stamp,booth,iso_lbs_run_usage,poly_lbs_run_usage,Time
8,235970,2020-04-01 05:28:26.620,2020-04-01 10:24:03.637,88,admin,0.000000,NaN,2020-04-01 10:36:14.707,2.0,1251.0,1500.0,0.433375
25,NaN,2020-05-13 13:17:40.847,2020-05-12 09:29:37.110,NaN,admin,0.000000,NaN,2020-05-13 15:57:40.003,3.0,1251.0,1500.0,0.395568
32,238227,2020-05-22 08:33:51.213,2020-05-22 08:49:44.667,NaN,admin,0.000000,NaN,2020-05-22 08:51:49.307,3.0,1251.0,1500.0,0.367878
34,NaN,2020-05-26 08:00:02.053,2020-05-21 06:35:17.310,NaN,admin,0.000000,NaN,2020-05-26 08:34:05.563,2.0,1251.0,1500.0,0.274506
36,NaN,2020-06-01 16:52:30.607,2020-05-29 12:59:14.857,30,admin,0.000000,NaN,2020-06-01 16:54:48.407,3.0,1251.0,1500.0,0.541144
...,...,...,...,...,...,...,...,...,...,...,...,...
63731,308243,2022-11-21 16:19:09.850,2022-11-21 16:35:26.347,~$$$$$$$$$$$$$$$$$$$A8L/307228,operator,1631.315430,1710.142822,2022-11-21 16:35:27.707,4.0,45491.2,52489.8,0.691277
63732,308237,2022-11-21 19:26:54.757,2022-11-21 19:40:46.457,~$$$$$$$$$$$$$$$$$$$A8L/307228,operator,1635.633667,1702.666992,2022-11-21 19:40:47.757,4.0,46563.7,53727.4,0.819982
63739,308044,2022-11-21 19:54:45.523,2022-11-21 19:41:35.850,NaN,operator,1947.791870,1656.806641,2022-11-21 20:18:57.897,2.0,237477.0,274012.0,0.820554
63809,308334,2022-11-22 07:35:12.677,2022-11-22 07:52:25.417,NaN,operator,2033.768311,1728.824951,2022-11-22 07:52:25.473,2.0,263594.0,304146.0,0.328072


In [25]:
df1.head(1)

,serial#,start_time,end_time,mix_module,operator,iso_press_avg,poly_press_avg,t_stamp,booth,iso_lbs_run_usage,poly_lbs_run_usage,Time
0,13456,2020-03-20 15:22:31.517,2020-03-20 15:37:35.463,88,admin,0.0,NaN,2020-03-20 15:37:36.070,3.0,1251.0,1500.0,0.651105


In [26]:
df1['iso_press_avg'].value_counts()

0.000000       1650
1358.332642       5
1796.693359       4
1725.686035       4
1764.366455       4
               ... 
2656.639404       1
2122.339111       1
1974.365112       1
1859.327759       1
2250.809082       1
Name: iso_press_avg, Length: 59557, dtype: int64

In [27]:
df1['poly_press_avg'].value_counts()

0.000000       1506
1307.376221       5
1380.175659       4
1679.962891       4
1578.235718       4
               ... 
1517.786377       1
1564.792358       1
1468.007812       1
1527.161621       1
2007.501343       1
Name: poly_press_avg, Length: 59365, dtype: int64

In [28]:
df1['iso_lbs_run_usage'].value_counts()

1251.000      110
37138.400      79
501.248        70
26324.100      66
34743.100      50
             ... 
168.564         1
163.250         1
133.838         1
205.723         1
211852.000      1
Name: iso_lbs_run_usage, Length: 61243, dtype: int64

# Cleaning Spray Info Data

In [29]:
df1_clean = df1.copy()

### Issue: Mix Module column should be removed. It has many null values
### Resolution: Drop mix_module column

In [30]:
df1_clean.drop('mix_module', axis=1, inplace = True)

### Test: Make sure column has been removed

In [31]:
df1_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63843 entries, 0 to 63842
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   serial#             63808 non-null  object        
 1   start_time          63752 non-null  datetime64[ns]
 2   end_time            63759 non-null  datetime64[ns]
 3   operator            63752 non-null  object        
 4   iso_press_avg       63648 non-null  float64       
 5   poly_press_avg      63478 non-null  float64       
 6   t_stamp             63757 non-null  datetime64[ns]
 7   booth               63757 non-null  float64       
 8   iso_lbs_run_usage   63051 non-null  float64       
 9   poly_lbs_run_usage  63051 non-null  float64       
 10  Time                63843 non-null  float64       
dtypes: datetime64[ns](3), float64(6), object(2)
memory usage: 5.4+ MB


### Issue: there are many duplicate serial#
### Resolution: Drop duplicate serial# but keep most recent

In [32]:
df1_clean = df1_clean.sort_values('end_time').drop_duplicates('serial#', keep='last')

### Test: Make sure duplicates are gone from dataframe

In [33]:
duplicate_serial = df1_clean[df1_clean.duplicated('serial#')]
duplicate_serial

,serial#,start_time,end_time,operator,iso_press_avg,poly_press_avg,t_stamp,booth,iso_lbs_run_usage,poly_lbs_run_usage,Time


### Issue: 
### Resolution: 

df1_clean.to_excel('Spray_Info_Clean.xlsx', index = False)

# Merging all spas and spray info into a single dataframe on serial#

In [34]:
df_merged = pd.merge(df_clean, df1_clean, on = 'serial#', how = 'left')

In [35]:
df_merged.head()

,serial#,model,CompleteDate,Time Description,start_time,end_time,operator,iso_press_avg,poly_press_avg,t_stamp,booth,iso_lbs_run_usage,poly_lbs_run_usage,Time
0,235970,X7,2020-03-03 10:36:00,Noon-6PM,2020-04-01 05:28:26.620,2020-04-01 10:24:03.637,admin,0.0,NaN,2020-04-01 10:36:14.707,2.0,1251.0,1500.0,0.433375
1,236746,A7,2020-04-03 08:29:00,Noon-6PM,2020-03-30 15:22:19.347,2020-03-30 16:36:38.323,admin,0.0,NaN,2020-03-30 16:36:39.033,3.0,1251.0,1500.0,0.692110
2,236730,A7,2020-04-03 09:28:00,6PM-Midnight,2020-03-31 06:08:12.733,2020-03-30 19:14:41.770,admin,0.0,NaN,2020-03-31 06:37:54.207,3.0,1251.0,1500.0,0.801872
3,236771,X6L,2020-04-06 06:23:00,6AM-Noon,2020-03-31 09:53:41.243,2020-03-31 06:52:04.443,admin,0.0,NaN,2020-03-31 09:56:50.683,3.0,1251.0,1500.0,0.286163
4,236743,R5L,2020-04-06 07:37:00,6AM-Noon,2020-03-31 15:09:14.990,2020-03-31 10:08:29.040,admin,0.0,NaN,2020-03-31 15:24:20.290,3.0,1251.0,1500.0,0.422558


In [36]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59994 entries, 0 to 59993
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   serial#             59994 non-null  object        
 1   model               59994 non-null  object        
 2   CompleteDate        59994 non-null  datetime64[ns]
 3   Time Description    59994 non-null  object        
 4   start_time          59987 non-null  datetime64[ns]
 5   end_time            59994 non-null  datetime64[ns]
 6   operator            59987 non-null  object        
 7   iso_press_avg       59897 non-null  float64       
 8   poly_press_avg      59765 non-null  float64       
 9   t_stamp             59987 non-null  datetime64[ns]
 10  booth               59987 non-null  float64       
 11  iso_lbs_run_usage   59324 non-null  float64       
 12  poly_lbs_run_usage  59324 non-null  float64       
 13  Time                59994 non-null  float64   

In [37]:
df_merged = df_merged.dropna()

In [38]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59100 entries, 36 to 59993
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   serial#             59100 non-null  object        
 1   model               59100 non-null  object        
 2   CompleteDate        59100 non-null  datetime64[ns]
 3   Time Description    59100 non-null  object        
 4   start_time          59100 non-null  datetime64[ns]
 5   end_time            59100 non-null  datetime64[ns]
 6   operator            59100 non-null  object        
 7   iso_press_avg       59100 non-null  float64       
 8   poly_press_avg      59100 non-null  float64       
 9   t_stamp             59100 non-null  datetime64[ns]
 10  booth               59100 non-null  float64       
 11  iso_lbs_run_usage   59100 non-null  float64       
 12  poly_lbs_run_usage  59100 non-null  float64       
 13  Time                59100 non-null  float64  

### Adding weather info to combined spa data

In [40]:
df_spa = pd.read_excel('spa_data.xlsx')

In [44]:
df_weather = pd.read_excel('Blister Returns.xlsx', sheet_name = 'Weather')

In [45]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 996 entries, 0 to 995
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Date                 996 non-null    datetime64[ns]
 1   Avg Temp             996 non-null    float64       
 2   Avg Dew Point        996 non-null    float64       
 3   Avg Humidity         996 non-null    float64       
 4   Avg Pressure         996 non-null    float64       
 5   Total Precipitation  996 non-null    float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 46.8 KB


In [46]:
df_weather.head()

,Date,Avg Temp,Avg Dew Point,Avg Humidity,Avg Pressure,Total Precipitation
0,2020-03-01,33.8,30.4,87.7,25.6,0.00
1,2020-03-02,34.5,21.0,59.6,25.8,0.23
2,2020-03-03,38.6,24.1,57.2,25.8,0.00
3,2020-03-04,44.5,28.5,56.6,25.9,0.00
4,2020-03-05,52.1,25.6,39.3,25.9,0.00


In [41]:
df_spa.head()

,serial#,blister,model,time_description,start_time,end_time,t_stamp,iso_press_avg,poly_press_avg,booth,iso_lbs_run_usage,poly_lbs_run_usage,spray_time
0,239329,0,X6L,Noon-6PM,2020-06-29 15:51:41.617,2020-06-29 15:49:18.933,2020-06-29 16:04:07.387,1515.681641,1455.528809,1,1150.3600,1725.5300,0.659247
1,239506,0,R7,6AM-Noon,2020-06-26 06:31:24.403,2020-06-26 06:15:31.813,2020-06-26 06:45:52.803,1793.341553,1738.000000,2,99.0839,148.6260,0.260785
2,239505,0,A7,6AM-Noon,2020-06-26 06:05:28.090,2020-06-26 06:18:47.770,2020-06-26 06:18:48.383,2019.696655,1658.000000,3,47.0481,70.5721,0.263053
3,239484,0,X8,6AM-Noon,2020-06-26 06:39:23.573,2020-06-26 06:55:54.897,2020-06-26 06:55:56.197,1614.548462,1665.000000,1,107.0660,161.0000,0.288830
4,239487,0,X7,6AM-Noon,2020-06-26 06:05:07.870,2020-06-26 06:19:49.387,2020-06-26 06:19:50.087,1659.422852,1706.000000,1,51.8829,78.0000,0.263766


In [42]:
df_spa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59100 entries, 0 to 59099
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   serial#             59100 non-null  int64         
 1   blister             59100 non-null  int64         
 2   model               59100 non-null  object        
 3   time_description    59100 non-null  object        
 4   start_time          59100 non-null  datetime64[ns]
 5   end_time            59100 non-null  datetime64[ns]
 6   t_stamp             59100 non-null  datetime64[ns]
 7   iso_press_avg       59100 non-null  float64       
 8   poly_press_avg      59100 non-null  float64       
 9   booth               59100 non-null  int64         
 10  iso_lbs_run_usage   59100 non-null  float64       
 11  poly_lbs_run_usage  59100 non-null  float64       
 12  spray_time          59100 non-null  float64       
dtypes: datetime64[ns](3), float64(5), int64(3), ob

In [49]:
df_spa['stamp_date'] = df_spa['t_stamp'].dt.date

In [50]:
df_spa.head()

,serial#,blister,model,time_description,start_time,end_time,t_stamp,iso_press_avg,poly_press_avg,booth,iso_lbs_run_usage,poly_lbs_run_usage,spray_time,stamp_date
0,239329,0,X6L,Noon-6PM,2020-06-29 15:51:41.617,2020-06-29 15:49:18.933,2020-06-29 16:04:07.387,1515.681641,1455.528809,1,1150.3600,1725.5300,0.659247,2020-06-29
1,239506,0,R7,6AM-Noon,2020-06-26 06:31:24.403,2020-06-26 06:15:31.813,2020-06-26 06:45:52.803,1793.341553,1738.000000,2,99.0839,148.6260,0.260785,2020-06-26
2,239505,0,A7,6AM-Noon,2020-06-26 06:05:28.090,2020-06-26 06:18:47.770,2020-06-26 06:18:48.383,2019.696655,1658.000000,3,47.0481,70.5721,0.263053,2020-06-26
3,239484,0,X8,6AM-Noon,2020-06-26 06:39:23.573,2020-06-26 06:55:54.897,2020-06-26 06:55:56.197,1614.548462,1665.000000,1,107.0660,161.0000,0.288830,2020-06-26
4,239487,0,X7,6AM-Noon,2020-06-26 06:05:07.870,2020-06-26 06:19:49.387,2020-06-26 06:19:50.087,1659.422852,1706.000000,1,51.8829,78.0000,0.263766,2020-06-26


In [55]:
L = df_weather.shape[0]
i = 0
for x in range(L):
    date = df_weather.iloc[i]['Date']
    df_spa.loc[(df_spa['stamp_date'] == date), "Avg Temp"] = df_weather.iloc[i]['Avg Temp']
    df_spa.loc[(df_spa['stamp_date'] == date), "Avg Dew Point"] = df_weather.iloc[i]['Avg Dew Point']
    df_spa.loc[(df_spa['stamp_date'] == date), "Avg Humidity"] = df_weather.iloc[i]['Avg Humidity']
    df_spa.loc[(df_spa['stamp_date'] == date), "Avg Pressure"] = df_weather.iloc[i]['Avg Pressure']
    df_spa.loc[(df_spa['stamp_date'] == date), "Total Precipitation"] = df_weather.iloc[i]['Total Precipitation']
    i += 1

C:\Users\tpeterson\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\ops\array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)
C:\Users\tpeterson\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\ops\array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)
C:\Users\tpeterson\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\ops\array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in ord

In [56]:
df_spa.head()

,serial#,blister,model,time_description,start_time,end_time,t_stamp,iso_press_avg,poly_press_avg,booth,iso_lbs_run_usage,poly_lbs_run_usage,spray_time,stamp_date,Avg Temp,Avg Dew Point,Avg Humidity,Avg Pressure,Total Precipitation
0,239329,0,X6L,Noon-6PM,2020-06-29 15:51:41.617,2020-06-29 15:49:18.933,2020-06-29 16:04:07.387,1515.681641,1455.528809,1,1150.3600,1725.5300,0.659247,2020-06-29,53.8,40.5,64.4,25.6,0.28
1,239506,0,R7,6AM-Noon,2020-06-26 06:31:24.403,2020-06-26 06:15:31.813,2020-06-26 06:45:52.803,1793.341553,1738.000000,2,99.0839,148.6260,0.260785,2020-06-26,78.4,43.8,33.6,25.7,0.00
2,239505,0,A7,6AM-Noon,2020-06-26 06:05:28.090,2020-06-26 06:18:47.770,2020-06-26 06:18:48.383,2019.696655,1658.000000,3,47.0481,70.5721,0.263053,2020-06-26,78.4,43.8,33.6,25.7,0.00
3,239484,0,X8,6AM-Noon,2020-06-26 06:39:23.573,2020-06-26 06:55:54.897,2020-06-26 06:55:56.197,1614.548462,1665.000000,1,107.0660,161.0000,0.288830,2020-06-26,78.4,43.8,33.6,25.7,0.00
4,239487,0,X7,6AM-Noon,2020-06-26 06:05:07.870,2020-06-26 06:19:49.387,2020-06-26 06:19:50.087,1659.422852,1706.000000,1,51.8829,78.0000,0.263766,2020-06-26,78.4,43.8,33.6,25.7,0.00


In [57]:
df_spa.to_excel('model_dataset.xlsx', index= False)

In [58]:
df_spa['blister'].value_counts(True)

0    0.98758
1    0.01242
Name: blister, dtype: float64